# Homework 2 : CUAHSI VU : Snow Hydrology

This notebook will be relatively short because this where the real work starts and most of the notebook will be written by you (or at least that is the plan). We'll do three exercises in this homework:

* Homework 1 exercise 1: Change the snow layering scheme

* Homework 1 exercise 2: Change the albedo formulation and albedo parameters

* Homework 1 exercise 3: Change the parameters controlling drainage through the snow pack


## Homework 1 Exercise 1

### Setup

Make sure that the CUES site is set up before you run this notebook (see `install_summa_model_configurations.ipynb`).

We'll import some of the python modules that you are likely to need right up front. Note that you only need to do this once in a notebook. As long as you are working in the same session, the modules will remain loaded for the duration of that session.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt # if you want to do some plotting using matplotlib (optional)
import pysumma as ps

We'll also set some variables to make sure that everyone is using the same setup

In [ ]:
executable = 'summa.exe'
filemanager = '/home/jovyan/data/cues/summa_setup/settings/file_manager_cues.txt'

### Change the snow layering scheme

Snow models represent a snow pack as one or more layers of snow and keep track of how much water is stored in each layer (snow water equivalent or SWE) as well as other state variables such as temperature. The model then tracks the time evolution of these state variables in each layer as well as the fluxes of water and energy between the layers.

In SUMMA you can choose between two different methods to combine and sub-divide snow layers. This behavior is controlled by the `snowLayers` variable in the [SUMMA decisions file](https://summa.readthedocs.io/en/latest/input_output/SUMMA_input/#infile_model_decisions) and can take one of two values, either `CLM_2010` or `jrdn1991`. The first refers to how snow layers were handled in the Community Land Model (CLM), while the second refers the snow layering scheme in SNTHERM as describe in  Jordan (1991).

Among others things, the `CLM_2010` only has a limited number of snow layers. The layers store a maximum amount of SWE and when SWE in a layer exceeds that value, a new layer is created. Similarly, when SWE in a layer falls below a minimum SWE, the layer is merged with a neighboring layer. Once the maximum number of layers has been created, the deepest layer can grow as deep as it needs to be and acts as snow reservoir with unlimited capacity. The `jrdn1991` scheme on the other hand keeps creating additional layers as the snow pack grows in depth.

Here we will do four different simulations:

* SUMMA with `CLM_2010`
* SUMMA with `jrdn1991`
* SUMMA with `CLM_2010`, but we will configure it to act as a two layer model (the default version uses five snow layers)
* SUMMA with `CLM_2010`, a two layer model and a thick top layer.

So now it is up to you. Just to refresh your memory we'll show you how to get started and then will provide some hints along the way.

#### SUMMA with CLM_2010

Create a simulation object. The path for the SUMMA code and the file manager (the SUMMA input file with information about all the other input files) are already defined above. We can use the same file manager file (but different simulation objects) for all the runs, just keep track of what you did.

In [ ]:
s_clm = ps.Simulation(executable, filemanager)

# Shorten the simulation length
s_clm.decisions['simulStart'] = '2010-10-01 00:00'
s_clm.decisions['simulFinsh'] = '2012-10-01 00:00'

# Add in some additional variables so we can demonstrate plotting capabilities
output_settings = {'period': 1, 'instant': 0, 'sum': 1, 
              'mean': 0, 'variance': 0, 'min': 0, 'max': 0}
layer_vars = ['mLayerTemp', 'mLayerDepth', 'mLayerHeight',
              'mLayerLiqFluxSoil', 'mLayerVolFracIce', 'mLayerVolFracLiq', 
              'mLayerVolFracWat','mLayerMatricHead', 'iLayerHeight', 'scalarSnowDepth']

# Create the new variables
for var in layer_vars:
    s_clm.output_control[var] = output_settings

# Ensure all variables have the same statistics
all_vars = set(layer_vars + [o.name for o in s_clm.output_control.options])
for var in all_vars:
    s_clm.output_control[var] = output_settings

Use a `run_suffix` that uniquely identifies this run so that you can keep your runs apart, for example `CLM_2010`, and run the model. If you assign the results to its own dataset (e.g. `ds_clm_2010`) and output file (e.g. `file_clm_2010`) it will be easier to gather all the files and upload them to HydroShare (or you can do so one by one, this is all up to you). The run itself will take about a minute.

In [ ]:
s_clm.run('local', run_suffix='CLM_2010')

In [ ]:
s_clm.output['nSnow'].sel(hru=1).plot()

As you can see, the number of layers never exceeds five.

#### SUMMA with jrdn1991

Repeat the exercise with the `jrdn1991` option for `snowLayers`. Examine the differences in the snow layers themselves, but also in other characteristics of snow pack evolution. In SUMMA, all other options will be the same between these two simulations unless you change things explicitly.

In [ ]:
s_jrdn = ps.Simulation(executable, filemanager)
s_jrdn.decisions['snowLayers'] = 'jrdn1991'

In [ ]:
s_jrdn.run('local', run_suffix='_jrdn')

In [ ]:
print(s_jrdn.status)

#### SUMMA with CLM_2010 configured as a  two layer model

Now change the `snowLayers` option back to `CLM_2010` and configure the model to act as a two layer snow model rather than a five layer snow model. Two layer models are quite common. The surface layer is typically quite thin in these setups to respond quickly to changing atmospheric conditions, while the second layer acts as a snow reservoir with unlimited capacity.

To configure the snow model as a two layer model, we need to change some model parameters. As in the pySUMMA tutorial, we can access and change these parameters with the `local_param_info` attribute of a `Simulation` object.


In [ ]:
s_2layer = ps.Simulation(executable, filemanager)

s_2layer.decisions['snowLayers'] = 'CLM_2010'

s_2layer.local_param_info['zminLayer3'] = 100.0000
s_2layer.local_param_info['zminLayer4'] = 100.0000
s_2layer.local_param_info['zminLayer5'] = 100.0000
s_2layer.local_param_info['zmaxLayer2_lower'] = 1000.0000
s_2layer.local_param_info['zmaxLayer3_lower'] = 1000.0000
s_2layer.local_param_info['zmaxLayer4_lower'] = 1000.0000
s_2layer.local_param_info['zmaxLayer2_upper'] = 1000.0000
s_2layer.local_param_info['zmaxLayer3_upper'] = 1000.0000
s_2layer.local_param_info['zmaxLayer4_upper'] = 1000.0000

Now go ahead and run the model. 

In [ ]:
s_2layer.run('local', run_suffix='_2layer')

In [ ]:
print(s_2layer.status)

#### SUMMA with CLM_2010 configured as a two layer model with a thick top layer

One more run with `CLM_2010`, two layers, and a thick top layer. To do this, change the local parameters to

In [ ]:
s_thick = ps.Simulation(executable, filemanager)

s_thick.decisions['snowLayers'] = 'CLM_2010'

s_thick.local_param_info['zminLayer1'] = 0.050
s_thick.local_param_info['zminLayer2'] = 0.020
s_thick.local_param_info['zminLayer3'] = 100.0
s_thick.local_param_info['zminLayer4'] = 100.0
s_thick.local_param_info['zminLayer5'] = 100.0
s_thick.local_param_info['zmaxLayer1_lower'] = 0.2000
s_thick.local_param_info['zmaxLayer2_lower'] = 1000.0
s_thick.local_param_info['zmaxLayer3_lower'] = 1000.0
s_thick.local_param_info['zmaxLayer4_lower'] = 1000.0
s_thick.local_param_info['zmaxLayer1_upper'] = 0.1500
s_thick.local_param_info['zmaxLayer2_upper'] = 1000.0
s_thick.local_param_info['zmaxLayer3_upper'] = 1000.0
s_thick.local_param_info['zmaxLayer4_upper'] = 1000.0

In [ ]:
s_thick.run('local', run_suffix='_thick')

In [ ]:
print(s_thick.status)

### Plotting

To examine how these changes impact the model output make some plots. Below is just an example comparing the amount of SWE across the four runs. You may wish to try some other plots, such as the layering plots described in the pySUMMA tutorial.

In [ ]:
s_clm.output['scalarSWE'].plot(label='CLM layering')
s_jrdn.output['scalarSWE'].plot(label='Jordan1991 layering')
s_2layer.output['scalarSWE'].plot(label='Thin 2 layer')
s_thick.output['scalarSWE'].plot(label='Thick 2 layer')
plt.legend()

After you are done, restore the local parameters to the original values, because we will use the same setup in the next notebook. This can be done with the following cell. Basically, we take the initial `s_clm` `Simulation` object and re-write it's configuration which overwrites any changes made by the other simulations we ran. If you try this and it doesn't seem to work, or things go really wrong some other way you can rerun the setup cell at the beginning of this notebook and start with a clean slate.

In [ ]:
s_clm._write_configuration()

## On to the next step

That's it for the first part of the first homework as far as the model simulations go. Prof. Lundquist will talk more about the analysis that you need to do on these simulations. You can do the analysis here in this notebook if python is your analysis software of choice or you can upload your output to HydroShare (in NetCDF or csv format) to do additional analysis.

Save this notebook and close the tab. You can also right-click on the file in the left panel if it has a green dot next to it and select "_Shutdown kernel_" from the popup menu to stop the python session that is executing the commands in this notebook. Then open the next notebook: 
`/home/jovyan/data/cuahsi_vu/homework_2/homework_2_B_exercise_2.ipynb`.

## References

* Jordan, R., 1991: A one-dimensional temperature model for a snow cover. Technical documentation for SNTHERM.89. U.S. Army Corps of Engineers Cold Regions Research and Engineering Laboratory, 49 pp.